In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import io
import xgboost as xgb
from sklearn.model_selection import train_test_split
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
warnings.filterwarnings('ignore')

In [ ]:
name_columns_train = ['periodo', 'id_cli', 'fecha_nacimiento', 'edad', 'genero', 'estado_civil', 'nivel_academico', 'profesion', 'ocupacion', 'tipo_vivienda', 'ult_actual', 'categoria', 'codigo_ciiu', 'ind_mora_vigente', 'cartera_castigada', 'ciudad_residencia', 'departamento_residencia', 'ciudad_laboral', 'departamento_laboral', 'rechazo_credito', 'mora_max', 'cant_moras_30_ult_12_meses', 'cant_moras_60_ult_12_meses', 'cant_moras_90_ult_12_meses', 'cupo_total_tc', 'tenencia_tc', 'cuota_tc_bancolombia', 'tiene_consumo', 'tiene_crediagil', 'nro_tot_cuentas', 'ctas_activas', 'tiene_ctas_activas', 'ctas_embargadas', 'tiene_ctas_embargadas', 'pension_fopep', 'cuota_cred_hipot', 'tiene_cred_hipo_1', 'tiene_cred_hipo_2', 'mediana_nom3', 'mediana_pen3', 'ingreso_nompen', 'cat_ingreso', 'ingreso_final', 'cant_mora_30_tdc_ult_3m_sf', 'cant_mora_30_consum_ult_3m_sf', 'cuota_de_vivienda', 'cuota_de_consumo', 'cuota_rotativos', 'cuota_tarjeta_de_credito', 'cuota_de_sector_solidario', 'cuota_sector_real_comercio', 'cupo_tc_mdo', 'saldo_prom3_tdc_mdo', 'cuota_tc_mdo', 'saldo_no_rot_mdo', 'cuota_libranza_sf', 'cant_oblig_tot_sf', 'cant_cast_ult_12m_sr', 'ind', 'rep_calif_cred', 'pol_centr_ext', 'convenio_lib', 'ingreso_nomina', 'ingreso_segurida_social', 'gasto_familiar']
columnas = ['edad','ingreso_final', 'ind', 'gasto_familiar']
df_train = pd.read_csv('/kaggle/input/datatontrain2020/Dataton_train.csv', error_bad_lines=False, names = name_columns_train, usecols = columnas)
test_data = pd.read_csv('/kaggle/input/datatonbc-2020/dt2020_base_evaluar/dt2020_base_evaluar.csv')

In [ ]:
df_train.info()

In [ ]:
df_train['edad'] = pd.to_numeric(df_train['edad'], downcast='integer', errors='coerce')
df_train['ingreso_final'] = pd.to_numeric(df_train['ingreso_final'], downcast='integer', errors='coerce')
df_train['gasto_familiar'] = pd.to_numeric(df_train['gasto_familiar'], downcast='integer', errors='coerce')
df_train['ind'] = pd.to_numeric(df_train['ind'], downcast='integer', errors='coerce')

In [ ]:
df_train_clean = df_train.copy()

In [ ]:
df_train_clean.info()

In [ ]:
df_train_clean['ingreso_final']= df_train_clean['ingreso_final'].fillna(df_train_clean['ingreso_final'].mean(axis = 0, skipna = True))
df_train_clean['edad']= df_train_clean['edad'].fillna(df_train_clean['edad'].mean(axis = 0, skipna = True))
df_train_clean['ind']= df_train_clean['ind'].fillna(df_train_clean['ind'].mean(axis = 0, skipna = True))
df_train_clean['gasto_familiar']= df_train_clean['gasto_familiar'].fillna(df_train_clean['gasto_familiar'].mean(axis = 0, skipna = True))

In [ ]:
max_val = df_train_clean.gasto_familiar.quantile(0.95)
min_val = df_train_clean.gasto_familiar.quantile(0.05)

max_val_ing = df_train_clean.ingreso_final.quantile(0.95)
min_val_ing = df_train_clean.ingreso_final.quantile(0.05)

max_val_ind = df_train_clean.ind.quantile(0.95)
min_val_ind = df_train_clean.ind.quantile(0.05)

In [ ]:
df_train_clean = df_train_clean[(df_train_clean['ingreso_final'] > min_val_ing) & (df_train_clean['ingreso_final']< max_val_ing)]
df_train_clean = df_train_clean[(df_train_clean['ind'] > min_val_ind) & (df_train_clean['ind']< max_val_ind)]
df_train_clean = df_train_clean[(df_train_clean['gasto_familiar'] > min_val) & (df_train_clean['gasto_familiar']< max_val)]

In [ ]:
df_train_clean.corr(method='pearson')

In [ ]:
df_train_clean.corr(method='pearson')

In [ ]:
X_train = df_train_clean[['ingreso_final', 'ind']].values
y_train = df_train_clean['gasto_familiar'].values
xgbtrain = xgb.DMatrix(X_train, y_train)

In [ ]:
# params = {
#     'max_depth': 10,
#     'subsample': 1,
#     'min_child_weight': 0.5,
#     'eta': 0.3,
#     'num_round': 1000,
#     'seed': 1,
#     'silent': 0,
#     'eval_metric': 'rmse'
# }

params = {
    'max_depth': 10,
    'subsample': 1,
    'min_child_weight': 0.5,
    'eta': 0.3,
    'seed': 1,
    'eval_metric': 'mae'
}

boost = xgb.train(params, xgbtrain)

In [ ]:
xgbpredict_test = xgb.DMatrix(test_data[['ingreso_final', 'ind']].values)

In [ ]:
another_test = boost.predict(xgbpredict_test)

In [ ]:
another_test

In [ ]:
registro = test_data.id_registro
diccionario = {'id_registro':registro, 'gasto_familiar': another_test}
submission = pd.DataFrame(data=diccionario)
submission.to_csv('JDA_values_maeF.csv',index=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train_clean[['ingreso_final', 'ind']], df_train_clean[['gasto_familiar']], test_size = 0.1, random_state = 0)

In [ ]:
xgbtrainsplit = xgb.DMatrix(X_train, y_train)

In [ ]:
params = {
    'max_depth': 6,
    'subsample': 1,
    'min_child_weight': 1,
    'eta': 0.3,
    'seed': 1,
    'eval_metric': 'mae'
}

boost = xgb.train(params, xgbtrainsplit)

In [ ]:
xgbpredict_test = xgb.DMatrix(X_test)
pred = boost.predict(xgbpredict_test)

In [ ]:
mae = mean_absolute_error(y_test.values, pred)
print(mae)

In [ ]:
mae = mean_absolute_error(y_test.values, pred)
print(mae)